### Generate a `DOCX` file with fake content
- Generate 1 `DOCX` file with fake content (generated by `Faker`).

In [ ]:
# Import the Faker class from faker package
from faker import Faker

# Import the file provider we want to use
from faker_file.providers.docx_file import DocxFileProvider

FAKER = Faker()  # Initialise Faker instance

FAKER.add_provider(DocxFileProvider)  # Register the DOCX file provider

file = FAKER.docx_file()  # Generate a DOCX file

# This is just a string-like value, with a relative path to the file
print(file)

# Note, that `file` is this case is an instance of either `StringValue`
# or `BytesValue` objects, which inherit from `str` and `bytes`
# respectively, but add meta data. Meta data is stored inside the `data`
# property (`Dict`). One of the common attributes of which (among all
# file providers) is the `filename`, which holds an absolute path to the
# generated file.
print(file.data["filename"])

# Another common attribute (although it's not available for all providers)
# is `content`, which holds the text used to generate the file with.
print(file.data["content"])

### Provide content manually
- Generate 1 `DOCX` file with developer defined content.

In [ ]:
# The text we want have in our generated DOCX file
TEXT = """
“The Queen of Hearts, she made some tarts,
    All on a summer day:
The Knave of Hearts, he stole those tarts,
    And took them quite away.”
"""

# Generate a DOCX file with the given text
file = FAKER.docx_file(content=TEXT)

print(file)
print(file.data["content"])

- Similarly, generate 1 `PNG` file

In [ ]:
from faker_file.providers.png_file import PngFileProvider

FAKER.add_provider(PngFileProvider)

file = FAKER.png_file()

print(file)

### Provide templated content

You can generate documents from pre-defined templates.

In [ ]:
# The template. All standard Faker providers are available.
TEMPLATE = """
{{date}} {{city}}, {{country}}

Hello {{name}},

{{text}}

Address: {{address}}

Best regards,

{{name}}
{{address}}
{{phone_number}}
"""

# Note, that wrap_chars_after will automatically insert newline 
# characters after the given number of characters in the single 
# line has been reached.
file = FAKER.pdf_file(content=TEMPLATE, wrap_chars_after=80)

print(file)
print(file.data["content"])

### Archive types
#### ZIP archive containing 5 TXT files
As you might have noticed, some archive types are also supported.
The created archive will contain 5 files in TXT format (defaults).

In [ ]:
from faker_file.providers.zip_file import ZipFileProvider

FAKER.add_provider(ZipFileProvider)

file = FAKER.zip_file()

print(file)
print(file.data)

#### ZIP archive containing 3 DOCX files with text generated from a template

In [ ]:
from faker_file.providers.helpers.inner import create_inner_docx_file

file = FAKER.zip_file(
    prefix="zzz",
    options={
        "count": 3,
        "create_inner_file_func": create_inner_docx_file,
        "create_inner_file_args": {
            "prefix": "xxx_",
            "content": TEMPLATE,
        },
        "directory": "yyy",
    },
)

print(file)
print(file.data)

#### Nested ZIP archive
And of course nested archives are supported too. Create a `ZIP` file which
contains 5 `ZIP` files which contain 5 `ZIP` files which contain 2 `DOCX`
files.

- 5 `ZIP` files in the `ZIP` archive.
- Content is generated dynamically.
- Prefix the filenames in archive with ``nested_level_1_``.
- Prefix the filename of the archive itself with ``nested_level_0_``.
- Each of the `ZIP` files inside the `ZIP` file in their turn contains 5 other `ZIP`
  files, prefixed with ``nested_level_2_``, which in their turn contain 2
  DOCX files.

In [ ]:
from faker_file.providers.helpers.inner import create_inner_zip_file

file = FAKER.zip_file(
    prefix="nested_level_0_",
    options={
        "create_inner_file_func": create_inner_zip_file,
        "create_inner_file_args": {
            "prefix": "nested_level_1_",
            "options": {
                "create_inner_file_func": create_inner_zip_file,
                "create_inner_file_args": {
                    "prefix": "nested_level_2_",
                    "options": {
                        "count": 2,
                        "create_inner_file_func": create_inner_docx_file,
                        "create_inner_file_args": {
                            "content": TEXT + "\n\n{{date}}",
                        },
                    },
                },
            },
        },
    },
)

print(file)
print(file.data)

It works similarly for `EML` files (using ``EmlFileProvider``).

In [ ]:
from faker_file.providers.eml_file import EmlFileProvider
from faker_file.providers.helpers.inner import create_inner_docx_file

FAKER.add_provider(EmlFileProvider)

file = FAKER.eml_file(
    prefix="zzz",
    content=TEMPLATE,
    options={
        "count": 3,
        "create_inner_file_func": create_inner_docx_file,
        "create_inner_file_args": {
            "prefix": "xxx_",
            "content": TEXT + "\n\n{{date}}",
        },
    },
)

print(file)
print(file.data)

#### Create a ZIP file with random variety of different file types within
- 50 files in the ZIP archive (limited to DOCX, EPUB and TXT types).
- Content is generated dynamically.
- Prefix the filename of the archive itself with zzz_archive_.
- Inside the ZIP, put all files in directory zzz.

In [ ]:
from faker import Faker
from faker_file.providers.helpers.inner import (
    create_inner_docx_file,
    create_inner_epub_file,
    create_inner_txt_file,
    fuzzy_choice_create_inner_file,
)
from faker_file.providers.zip_file import ZipFileProvider
from faker_file.storages.filesystem import FileSystemStorage

FAKER = Faker()
STORAGE = FileSystemStorage()

kwargs = {"storage": STORAGE, "generator": FAKER}
file = ZipFileProvider(FAKER).zip_file(
    prefix="zzz_archive_",
    options={
        "count": 50,
        "create_inner_file_func": fuzzy_choice_create_inner_file,
        "create_inner_file_args": {
            "func_choices": [
                (create_inner_docx_file, kwargs),
                (create_inner_epub_file, kwargs),
                (create_inner_txt_file, kwargs),
            ],
        },
        "directory": "zzz",
    },
)

print(file)
print(file.data)

#### Another way to create a ZIP file with fixed variety of different file types within
- 3 files in the ZIP archive (1 DOCX, and 2 XML types).
- Content is generated dynamically.
- Filename of the archive itself is alice-looking-through-the-glass.zip.
- Files inside the archive have fixed name (passed with basename argument).

In [ ]:
from faker import Faker
from faker_file.providers.helpers.inner import (
    create_inner_docx_file,
    create_inner_xml_file,
    list_create_inner_file,
)
from faker_file.providers.zip_file import ZipFileProvider
from faker_file.storages.filesystem import FileSystemStorage

FAKER = Faker()
STORAGE = FileSystemStorage()

kwargs = {"storage": STORAGE, "generator": FAKER}
file = ZipFileProvider(FAKER).zip_file(
    basename="alice-looking-through-the-glass",
    options={
        "create_inner_file_func": list_create_inner_file,
        "create_inner_file_args": {
            "func_list": [
                (create_inner_docx_file, {"basename": "doc"}),
                (create_inner_xml_file, {"basename": "doc_metadata"}),
                (create_inner_xml_file, {"basename": "doc_isbn"}),
            ],
        },
    },
)

print(file)
print(file.data)

#### Using raw=True features in tests
If you pass raw=True argument to any provider or inner function, instead of creating a file, you will get bytes back (or to be totally correct, bytes-like object BytesValue, which is basically bytes enriched with meta-data). You could then use the bytes content of the file to build a test payload as shown in the example test below:

In [ ]:
import os
from io import BytesIO

from django.test import TestCase
from django.urls import reverse
from faker import Faker
from faker_file.providers.docx_file import DocxFileProvider
from rest_framework.status import HTTP_201_CREATED
from upload.models import Upload

FAKER = Faker()
FAKER.add_provider(DocxFileProvider)

class UploadTestCase(TestCase):
    """Upload test case."""

    def test_create_docx_upload(self) -> None:
        """Test create an Upload."""
        url = reverse("api:upload-list")

        raw = FAKER.docx_file(raw=True)
        test_file = BytesIO(raw)
        test_file.name = os.path.basename(raw.data["filename"])

        payload = {
            "name": FAKER.word(),
            "description": FAKER.paragraph(),
            "file": test_file,
        }

        response = self.client.post(url, payload, format="json")

        # Test if request is handled properly (HTTP 201)
        self.assertEqual(response.status_code, HTTP_201_CREATED)

        test_upload = Upload.objects.get(id=response.data["id"])

        # Test if the name is properly recorded
        self.assertEqual(str(test_upload.name), payload["name"])

        # Test if file name recorded properly
        self.assertEqual(str(test_upload.file.name), test_file.name)

### Create a HTML file predefined template
If you want to generate a file in a format that is not (yet) supported, you can try to use `GenericFileProvider`. In the following example, an HTML file is generated from a template.

In [ ]:
from faker import Faker
from faker_file.providers.generic_file import GenericFileProvider

file = GenericFileProvider(Faker()).generic_file(
    content="<html><body><p>{{text}}</p></body></html>",
    extension="html",
)

print(file)
print(file.data)

### Storages

#### Example usage with `Django` (using local file system storage)

In [ ]:
from django.conf import settings
from faker_file.providers.txt_file import TxtFileProvider
from faker_file.storages.filesystem import FileSystemStorage

STORAGE = FileSystemStorage(
    root_path=settings.MEDIA_ROOT,
    rel_path="tmp",
)

FAKER.add_provider(TxtFileProvider)

file = FAKER.txt_file(content=TEXT, storage=STORAGE)

print(file)
print(file.data["filename"])
print(file.data["content"])

#### Example usage with AWS S3 storage

In [ ]:
from faker_file.storages.aws_s3 import AWSS3Storage

S3_STORAGE = AWSS3Storage(
    bucket_name="artur-testing-1",
    root_path="tmp",  # Optional
    rel_path="sub-tmp",  # Optional
    # Credentials are optional too. If your AWS credentials are properly
    # set in the ~/.aws/credentials, you don't need to send them
    # explicitly.
    # credentials={
    #     "key_id": "YOUR KEY ID",
    #     "key_secret": "YOUR KEY SECRET"
    # },
)

file = FAKER.txt_file(storage=S3_STORAGE)

print(file)
print(file.data["filename"])
print(file.data["content"])

### Augment existing files
If you think `Faker` generated data doesn't make sense for you and you want
your files to look like a collection of 100 files you already have, you could
use augmentation features.

In [ ]:
from faker_file.providers.augment_file_from_dir import (
    AugmentFileFromDirProvider,
)

FAKER.add_provider(AugmentFileFromDirProvider)

file = FAKER.augment_file_from_dir(
    source_dir_path="/home/me/Documents/faker_file_source/",
    wrap_chars_after=120,
)

print(file)
print(file.data["filename"])
print(file.data["content"])

### CLI
Even if you're not using automated testing, but still want to quickly generate a file with fake content, you could use faker-file:

In [ ]:
  !faker-file generate-completion
  !source ~/faker_file_completion.sh

#### Generate an MP3 file:

In [ ]:
!faker-file mp3_file --prefix=my_file_

#### Generate 10 DOCX files:

In [ ]:
!faker-file docx_file --nb_files 10 --prefix=my_file_